In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
BEST_ACC = 48

Using TensorFlow backend.


In [2]:
def W_init(shape, name=None):
    values = rng.normal(loc=0, scale=1e-2,size=shape)
    return K.variable(values, name=name)
def b_init(shape, name=None):
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [3]:
input_shape=(105,105,1)
left_input = Input(input_shape)
right_input = Input(input_shape)

model = Sequential()
model.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
model.add(MaxPooling2D())
model.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
model.add(MaxPooling2D())
model.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
model.add(MaxPooling2D())
model.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
model.add(Flatten())
model.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

encoded_l = model(left_input)
encoded_r = model(right_input)

L1_dist = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_dist, output_shape = lambda x: x[0])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)

optimizer = Adam(0.00006)
siamese_net.compile(loss='binary_crossentropy',optimizer=optimizer)

siamese_net.count_params()


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


38951745

In [5]:
class Siamese_Loader:
    def __init__(self,path):
        self.data = {}
        self.classes = {}
        with open(os.path.join(path,"train.pickle"), "rb") as f:
            (X,c) = pickle.load(f)
            self.data["train"] = X
            self.classes["train"] = c
        with open(os.path.join(path,"val.pickle"), "rb") as f:
            (X,c) = pickle.load(f)
            self.data["val"] = X
            self.classes["val"] = c
        self.n_classes,self.n_examples,self.w,self.h = self.data['train'].shape
#         n_classes: ennyi betu van a tanulo adatban (964)
#         n_examples: egy betuhoz 20 minta tartozik
#         w: a kepek szelessege (105)
#         h: a kepek magassaga (105)
        self.n_val,self.n_ex,_,_ = self.data['val'].shape
    
    '''Oszlopvektorokat alakit ki a kepekbol
    - a parok masodik fele megegyezo osztalybol az elso fele kulonbozo osztalybol szarmazik
    '''
    def get_batch(self,n,s='train'):
        X=self.data[s]
        classes=rng.choice(self.n_classes,size=(n,),replace=False)
        pairs=[np.zeros((n,self.h,self.w,1)) for i in range(2)]
        targets=np.zeros((n,))
        targets[n//2:] = 1
        for i in range(n):
            class_ = classes[i]
            idx_1 = rng.randint(0,self.n_examples)
            pairs[0][i,:,:,:] = X[class_,idx_1].reshape(self.w,self.h,1)
            idx_2 = rng.randint(0,self.n_examples)
            class_2 = class_ if i >= n//2 else (class_ + rng.randint(1,self.n_classes)) % self.n_classes 
            pairs[1][i,:,:,:] = X[class_2,idx_2].reshape(self.w,self.h,1)
        return pairs, targets
    
    def do_oneshot_task(self, N,s='val', language=None):
        X = self.data[s]
        n_classes,n_examples = X.shape[0],X.shape[1]
        if language is None:
            classes = rng.choice(range(n_classes),size=(N,),replace=False)
            '''Nem biztos, hogy kell de szerintem igy a jo'''
            #             start_idx,end_idx = self.classes[s]
#             start_idx = 0
#             end_idx = self.classes[n].shape[0]
            indicies = rng.randint(0,self.n_examples,size=(N,))
        else:
            start_idx,end_idx = self.classes[s][language]
            try:
                classes = rng.choice(range(start_idx,end_idx),size=(N,),replace=False)
            except ValueError:
                    print("This language doesn't have enough characters for that one-shot task")
            indicies = rng.randint(0,self.n_examples,size=(N,))
        true_class = classes[0]
        ex1,ex2 = rng.choice(n_examples,size=(2,),replace=False)
#         A teszt kepet lemasoljuk sokszor, hogy a siamese halo egyik agara mindig ezt tegyuk 
        test_image = np.asarray([X[true_class,ex1,:,:]]*N).reshape(N,self.w,self.h,1)
        support_set = X[classes,indicies,:,:]
        support_set[0,:,:] = X[true_class,ex2]
        support_set = support_set.reshape(N,self.w,self.h,1)
        targets = np.zeros((N,))
        targets[0] = 1
        targets,test_image,support_set=shuffle(targets,test_image,support_set)
        pairs = [test_image,support_set]
        return pairs, targets
    
    '''Az N azt mondja meg, hogy egy kepet hany masikkal vizsgalunk meg'''
    '''A k azt mondja meg, hogy hanyszor teszteljuk a halot "hany one shot feladatot vegezzunk el"'''
    def test_oneshot(self,model,N,k,s='val',verbose=0):
        n_correct = 0
        if verbose:
            print("Evaluating model on {} unique {} way one-shot learning tasks ...".format(k,N))
        for i in range(k):
            inputs,targets = self.do_oneshot_task(N,s,language=None)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
        percent_correct = (100*n_correct / k)
        if verbose:
            print("Got an avarage of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct

loader = Siamese_Loader("/Users/krisz/Documents/University/Szakdoga/one-shot")

In [6]:
pairs,targets = loader.do_oneshot_task(25, "train",language="Anglo-Saxon_Futhorc")

In [7]:
evaluate_every = 1
loss_every = 1
batch_size = 32
N_way = 4
n_val = 50

siamese_net.load_weights("/Users/krisz/Documents/University/Szakdoga/one-shot/weights")

for i in range(0, 201):
    (inputs,targets) = loader.get_batch(batch_size)
    loss = siamese_net.train_on_batch(inputs,targets)
    if i % evaluate_every == 0:
        val_acc = loader.test_oneshot(siamese_net,N_way,n_val,verbose=True)
        if val_acc >= BEST_ACC:
            print("saving...")
            siamese_net.save('/Users/krisz/Documents/University/Szakdoga/one-shot/weights') 
            BEST_ACC = val_acc
        if i % loss_every == 0:
            print("Iteration {}, training loss: {:.2f},".format(i,loss))

            

Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 70% 4 way one-shot learning accuracy
saving...
Iteration 0, training loss: 2.89,
Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 70% 4 way one-shot learning accuracy
saving...
Iteration 1, training loss: 2.82,
Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 78% 4 way one-shot learning accuracy
saving...
Iteration 2, training loss: 2.81,
Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 74% 4 way one-shot learning accuracy
Iteration 3, training loss: 2.92,
Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 62% 4 way one-shot learning accuracy
Iteration 4, training loss: 2.80,
Evaluating model on 50 unique 4 way one-shot learning tasks ...
Got an avarage of 76% 4 way one-shot learning accuracy
Iteration 5, training loss: 2.82,
Evaluating model on 50 unique 4 way one-shot learnin

KeyboardInterrupt: 

In [8]:
BEST_ACC

44